In [27]:
import pandas as pd
import spacy
nlp = spacy.load(name='en_core_web_sm')

df = pd.read_csv('./data/train-bio.csv', sep='\t', header=None)
df.columns = ['tokens','BIO_Label']

In [ ]:
import pandas as pd
import numpy as np
import re
import pickle 
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
from sklearn import metrics
from sklearn.metrics import confusion_matrix
#nltk.download('stopwords')

X,y = df['tokens'],df['BIO_Label']

corpus = []
for i in range(len(X)):
    review = re.sub(r'\W', ' ', str(X[i]))
    review = review.lower()
    review = re.sub(r'\s+[a-z]\s+', ' ',review) #substitute all single characters with space on l&r
    review = re.sub(r'\s+', ' ', review)  #substitute all spaces to single space
    corpus.append(review) 

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 2000, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
text_train, text_test, sent_train, sent_test = train_test_split(X[0:2000], y[0:2000], test_size = 0.20, random_state = 0)


In [12]:
import spacy
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import pandas as pd
import numpy as np
import re
import pickle 
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
nlp = spacy.load('en_core_web_lg')

df_shuffled = df.sample(frac=1).reset_index(drop=True) #shuffle your dataframe in-place and reset the index, only sample will also work

df_shuffled = df_shuffled.iloc[:3000]

for token,biolabel in df_shuffled.itertuples(index=False):
    if str(token) not in stopwords.words('english'):
        token = re.sub(r'\W', ' ', str(token))
        token = token.lower()
        token = re.sub(r'\s+[a-z]\s+', ' ',token) #substitute all single characters with space on l&r
        token = re.sub(r'\s+', ' ', token)  #substitute all spaces to single space
        # we have to remove empty token which are being generated by substituting

df_shuffled.isnull().sum() 
df_shuffled.dropna(inplace=True)

def get_vec(x):
    doc = nlp(str(x))
    vec = doc.vector
    return vec

df_shuffled['vec'] = df_shuffled['tokens'].apply(lambda x: get_vec(x))

X = df_shuffled['vec'].to_numpy()
X = X.reshape(-1, 1)

X = np.concatenate(np.concatenate(X, axis = 0), axis = 0).reshape(-1, 300)

y = df_shuffled['BIO_Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

# clf = LinearSVC()
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))


svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)
svm_predictions = svm_model_linear.predict(X_test)
accuracy = svm_model_linear.score(X_test, y_test)
print(accuracy)
print(metrics.classification_report(y_test, svm_predictions))
cm = confusion_matrix(y_test, svm_predictions)

# dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train)
# dtree_predictions = dtree_model.predict(X_test)
# accuracy = dtree_model.score(X_test, y_test)
# print(accuracy)
# print(metrics.classification_report(y_test, dtree_predictions))
# cm = confusion_matrix(y_test, dtree_predictions)

# naive_bayes_model = GaussianNB().fit(X_train, y_train)
# naive_bayes_predictions = naive_bayes_model.predict(X_test)
# accuracy = naive_bayes_model.score(X_test, y_test)
# print(accuracy)
# print(metrics.classification_report(y_test, naive_bayes_predictions))
# cm = confusion_matrix(y_test, naive_bayes_predictions)


df2 = pd.DataFrame(cm,index=['B','I','O'], columns=['B','I','O'])
print(df2)


0.6783333333333333
              precision    recall  f1-score   support

           B       0.00      0.00      0.00        23
           I       0.73      0.86      0.79       398
           O       0.55      0.36      0.43       179

    accuracy                           0.68       600
   macro avg       0.43      0.41      0.41       600
weighted avg       0.65      0.68      0.65       600

   B    I   O
B  0   19   4
I  6  343  49
O  6  109  64


In [57]:
df_shuffled.head()

,tokens,BIO_Label,vec
0,mobile,I,"[-0.14581, 0.36688, 0.31404, 0.33013, 0.55531,..."
1,not,I,"[-0.04983, 0.02705, -0.38787, -0.25857, -0.019..."
2,to,I,"[0.31924, 0.06316, -0.27858, 0.2612, 0.079248,..."
3,doubt,O,"[-0.1683, 0.070158, -0.20011, -0.16466, -0.054..."
4,community,I,"[0.2045, 0.22617, 0.040692, 0.00032489, 0.2187..."


In [ ]:
#here we will predict for test-bio
import pandas as pd
df_abc = pd.read_csv('./data/test-bio.csv', sep='\t', header=None)
df_abc.columns = ['tokens','BIO_Label']

df_abc.head()

svm_predictions = svm_model_linear.predict('sahil')
# print(metrics.classification_report(df_abc['BIO_Label'], svm_predictions))
# cm = confusion_matrix(df_abc['BIO_Label'], svm_predictions)
# print(cm)

# accuracy = svm_model_linear.score(X_test, y_test)
# print(accuracy)
# print(metrics.classification_report(y_test, svm_predictions))
# cm = confusion_matrix(y_test, svm_predictions)